In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf

In [2]:
### IMPORTS

import numpy as np 
import pandas as pd 
import random

import cv2
import os

import tensorflow as tf
from tensorflow import keras

from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout

In [3]:
pos = os.listdir('Data/Training/BrickImages')
neg =  os.listdir('Data/Training/NoBrickImages')
neg = ['Data/Training/NoBrickImages/'+ i for i in neg]
pos = ['Data/Training/BrickImages/'+ i for i in pos]
print("Negative labels: ",len(neg))
print("Positive labels: ",len(pos))

Negative labels:  11277
Positive labels:  15864


In [8]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0/255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range= 10,
    vertical_flip= True
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0/255, 
)

In [10]:
train_ds = train_gen.flow_from_directory(
        'Data/Training',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
)

test_ds = test_gen.flow_from_directory(
        'Data/Validation',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
)

Found 27140 images belonging to 2 classes.
Found 3280 images belonging to 2 classes.


In [11]:
### CREATE SEQUENTIAL MODEL

input_shape = (64, 64, 3)
print (input_shape)
print('Creating Model')

model = Sequential()

#filters,kernel_size,strides=(1, 1),padding='valid',data_format=None,dilation_rate=(1, 1),activation=None,use_bias=True,
#kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,
#activity_regularizer=None,kernel_constraint=None,bias_constraint=None,

#pool_size=(2, 2), strides=None, padding='valid',data_format=None

model.add(Conv2D(32, (3,3),padding='same',input_shape=input_shape,name='conv2d_1'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_1'))

model.add(Conv2D(64, 3, 3, name='conv2d_6'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
#model.add(Dropout(0.5))
model.add(Dense(1)) #config.num_classes
model.add(Activation('sigmoid'))

(64, 64, 3)
Creating Model


In [12]:
### COMPILE MODEL

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [13]:
### TRAIN MODEL

model.fit(
        train_ds,
        epochs=5, #config.num_epochs
        validation_data=test_ds, 

)

Epoch 1/10
849/849 [==============================] - 374s 441ms/step - loss: 0.0986 - accuracy: 0.9696 - val_loss: 0.0580 - val_accuracy: 0.9802
Epoch 2/10
849/849 [==============================] - 50s 59ms/step - loss: 0.0845 - accuracy: 0.9745 - val_loss: 0.0555 - val_accuracy: 0.9838
Epoch 3/10
849/849 [==============================] - 64s 75ms/step - loss: 0.0823 - accuracy: 0.9742 - val_loss: 0.0622 - val_accuracy: 0.9820
Epoch 4/10
849/849 [==============================] - 66s 78ms/step - loss: 0.0852 - accuracy: 0.9746 - val_loss: 0.0773 - val_accuracy: 0.9845
Epoch 5/10
849/849 [==============================] - 69s 81ms/step - loss: 0.0780 - accuracy: 0.9748 - val_loss: 0.0651 - val_accuracy: 0.9765
Epoch 6/10
849/849 [==============================] - 66s 78ms/step - loss: 0.0770 - accuracy: 0.9759 - val_loss: 0.0668 - val_accuracy: 0.9790
Epoch 7/10
849/849 [==============================] - 64s 75ms/step - loss: 0.0773 - accuracy: 0.9757 - val_loss: 0.0725 - val_accurac

In [14]:
model_json = model.to_json()
with open("Models/smally_rack_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Models/smally_rack_model.h5")
print("Saved model to disk")

Saved model to disk
